In [7]:
def query(gene_type = 'gbM', modification = 'H2AZ'):

    return f"""
    with genes_chr1 as (Select start, "end", strand, chromosome from genes where type = '{gene_type}' and chromosome = 1),
genes_chr2 as (Select start, "end", strand, chromosome from genes where type = '{gene_type}' and chromosome = 2),
genes_chr3 as (Select start, "end", strand, chromosome from genes where type = '{gene_type}' and chromosome = 3),
genes_chr4 as (Select start, "end", strand, chromosome from genes where type = '{gene_type}' and chromosome = 4),
genes_chr5 as (Select start, "end", strand, chromosome from genes where type = '{gene_type}' and chromosome = 5),

percentiles_chr1 as (
select Count(*) as sites,
        FLOOR(
            (
                CASE
                    WHEN a.strand = '+' then (c.start - a.start)
                    else (a.end - c."end"
                )
            end * 100
        ) / (a.end - a.start
)
) as percentile
from histone_mods c, genes_chr1 a WHERE 
    c.chromosome = 1
    and c.modification = '{modification}'
and c.start + (a.end-a.start) >= a.start
and c."end" - (a.end -a.start) <= a."end"
group by percentile order by percentile),

percentiles_chr2 as (
select Count(*) as sites,
        FLOOR(
            (
                CASE
                    WHEN a.strand = '+' then (c.start - a.start)
                    else (a.end - c."end"
                )
            end * 100
        ) / (a.end - a.start
)
) as percentile
from histone_mods c, genes_chr2 a WHERE 
    c.chromosome = 2
    and c.modification = '{modification}'
and c.start + (a.end-a.start) >= a.start
and c."end" - (a.end -a.start) <= a."end"
group by percentile order by percentile),

percentiles_chr3 as(
select Count(*) as sites,
        FLOOR(
            (
                CASE
                    WHEN a.strand = '+' then (c.start - a.start)
                    else (a.end - c."end"
                )
            end * 100
        ) / (a.end - a.start
)
) as percentile
from histone_mods c, genes_chr3 a WHERE 
    c.chromosome = 3
    and c.modification = '{modification}'
and c.start + (a.end-a.start) >= a.start
and c."end" - (a.end -a.start) <= a."end"
group by percentile order by percentile),

percentiles_chr4 as (
select Count(*) as sites,
        FLOOR(
            (
                CASE
                    WHEN a.strand = '+' then (c.start - a.start)
                    else (a.end - c."end"
                )
            end * 100
        ) / (a.end - a.start
)
) as percentile
from histone_mods c, genes_chr4 a WHERE 
    c.chromosome = 4
    and c.modification = '{modification}'
and c.start + (a.end-a.start) >= a.start
and c."end" - (a.end -a.start) <= a."end"
group by percentile order by percentile),

percentiles_chr5 as (
select Count(*) as sites,
        FLOOR(
            (
                CASE
                    WHEN a.strand = '+' then (c.start - a.start)
                    else (a.end - c."end"
                )
            end * 100
        ) / (a.end - a.start
)
) as percentile
from histone_mods c, genes_chr5 a WHERE 
    c.chromosome = 5
    and c.modification = '{modification}'
and c.start + (a.end-a.start) >= a.start
and c."end" - (a.end -a.start) <= a."end"
group by percentile order by percentile)

select percentiles_chr1.sites + percentiles_chr2.sites + percentiles_chr3.sites + percentiles_chr4.sites + percentiles_chr5.sites as sites, percentiles_chr1.percentile as percentile
from percentiles_chr1, percentiles_chr2, percentiles_chr3, percentiles_chr4, percentiles_chr5
where  
percentiles_chr1.percentile = percentiles_chr2.percentile and
percentiles_chr1.percentile = percentiles_chr3.percentile and
percentiles_chr1.percentile = percentiles_chr4.percentile and
percentiles_chr1.percentile = percentiles_chr5.percentile
order by percentile
    """


In [3]:
query("UM", "H3K27Me3")

'\n    with genes_chr1 as (Select start, "end", strand, chromosome from genes where type = UM and chromosome = 1),\ngenes_chr2 as (Select start, "end", strand, chromosome from genes where type = UM and chromosome = 2),\ngenes_chr3 as (Select start, "end", strand, chromosome from genes where type = UM and chromosome = 3),\ngenes_chr4 as (Select start, "end", strand, chromosome from genes where type = UM and chromosome = 4),\ngenes_chr5 as (Select start, "end", strand, chromosome from genes where type = UM and chromosome = 5),\n\npercentiles_chr1 as (\nselect Count(*) as sites,\n        FLOOR(\n            (\n                CASE\n                    WHEN a.strand = \'+\' then (c.start - a.start)\n                    else (a.end - c."end"\n                )\n            end * 100\n        ) / (a.end - a.start\n)\n) as percentile\nfrom histone_mods c, genes_chr1 a WHERE \n    c.chromosome = 1\n    and c.modification = H3K27Me3\nand c.start + (a.end-a.start) >= a.start\nand c."end" - (a.en

In [ ]:
import clickhouse_connect
import polars as pl
client = clickhouse_connect.get_client(host='localhost', username='cgoeldel', password='Goe1409ldel')

results = {}

for gene_type in ["UM", "teM", "gbM"]:
    for modification in ["H2AZ","H3","H3K36Me3","H3K4Me1","H3K27Me3","H3K4Me3","H3K56Ac","input"]:
        res = client.query_arrow(query(gene_type, modification))
        df = pl.DataFrame(res)
        df.write_csv(f"/mnt/fast/epigenomics/modifications/percentiles/{gene_type}_{modification}.csv")
        results[f"{gene_type}_{modification}"] = df
        




In [26]:
def chr_state_query(gene_type='gbM', state = 1):
    return f"""
    with genes_chr1 as (Select start, "end", strand, chromosome from genes where type = '{gene_type}' and chromosome = 1),
genes_chr2 as (Select start, "end", strand, chromosome from genes where type = '{gene_type}' and chromosome = 2),
genes_chr3 as (Select start, "end", strand, chromosome from genes where type = '{gene_type}' and chromosome = 3),
genes_chr4 as (Select start, "end", strand, chromosome from genes where type = '{gene_type}' and chromosome = 4),
genes_chr5 as (Select start, "end", strand, chromosome from genes where type = '{gene_type}' and chromosome = 5),

percentiles_chr1 as (
select Count(*) as sites,
        FLOOR(
            (
                CASE
                    WHEN a.strand = '+' then ((c.start + c.end) / 2 - a.start)
                    ELSE (a.end - (c.start + c."end") / 2) END * 100
        ) / (a.end - a.start)
) as percentile
from chr_states c, genes_chr1 a WHERE 
    c.chromosome = 1
    and c.state = '{state}'
and (c.start + c.end) / 2 + (a.end- a.start) >= a.start
and (c.start + c.end) / 2 - (a.end -a.start) <= a."end"
group by percentile order by percentile),

percentiles_chr2 as (
select Count(*) as sites,
        FLOOR(
            (
                CASE
                    WHEN a.strand = '+' then ((c.start + c.end) / 2 - a.start)
                    ELSE (a.end - (c.start + c."end") / 2) END * 100
        ) / (a.end - a.start)
) as percentile
from chr_states c, genes_chr2 a WHERE 
    c.chromosome = 2
    and c.state = '{state}'
and (c.start + c.end) / 2 + (a.end- a.start) >= a.start
and (c.start + c.end) / 2 - (a.end -a.start) <= a."end"
group by percentile order by percentile),

percentiles_chr3 as(
select Count(*) as sites,
        FLOOR(
            (
                CASE
                    WHEN a.strand = '+' then ((c.start + c.end) / 2 - a.start)
                    ELSE (a.end - (c.start + c."end") / 2) END * 100
        ) / (a.end - a.start)
) as percentile
from chr_states c, genes_chr3 a WHERE 
    c.chromosome = 3
    and c.state = '{state}'
and (c.start + c.end) / 2 + (a.end- a.start) >= a.start
and (c.start + c.end) / 2 - (a.end -a.start) <= a."end"
group by percentile order by percentile),

percentiles_chr4 as (
select Count(*) as sites,
        FLOOR(
            (
                CASE
                    WHEN a.strand = '+' then ((c.start + c.end) / 2 - a.start)
                    ELSE (a.end - (c.start + c."end") / 2) END * 100
        ) / (a.end - a.start)
) as percentile
from chr_states c, genes_chr4 a WHERE 
    c.chromosome = 4
    and c.state = '{state}'
and (c.start + c.end) / 2 + (a.end- a.start) >= a.start
and (c.start + c.end) / 2 - (a.end -a.start) <= a."end"
group by percentile order by percentile),

percentiles_chr5 as (
select Count(*) as sites,
        FLOOR(
            (
                CASE
                    WHEN a.strand = '+' then ((c.start + c.end) / 2 - a.start)
                    ELSE (a.end - (c.start + c."end") / 2) END * 100
        ) / (a.end - a.start)
) as percentile
from chr_states c, genes_chr5 a WHERE 
    c.chromosome = 5
    and c.state = '{state}'
and (c.start + c.end) / 2 + (a.end- a.start) >= a.start
and (c.start + c.end) / 2 - (a.end -a.start) <= a."end"
group by percentile order by percentile)

select percentiles_chr1.sites + percentiles_chr2.sites + percentiles_chr3.sites + percentiles_chr4.sites + percentiles_chr5.sites as sites, percentiles_chr1.percentile as percentile
from percentiles_chr1, percentiles_chr2, percentiles_chr3, percentiles_chr4, percentiles_chr5
where  
percentiles_chr1.percentile = percentiles_chr2.percentile and
percentiles_chr1.percentile = percentiles_chr3.percentile and
percentiles_chr1.percentile = percentiles_chr4.percentile and
percentiles_chr1.percentile = percentiles_chr5.percentile
order by percentile
    """


In [27]:
import clickhouse_connect
import polars as pl
client = clickhouse_connect.get_client(host='localhost', username='cgoeldel', password='Goe1409ldel')

chr_results = {}

for gene_type in ["UM", "teM", "gbM"]:
    for state in range(1, 36 + 1):
        res = client.query_arrow(chr_state_query(gene_type, state))
        df = pl.DataFrame(res)
        df.write_csv(f"/mnt/fast/epigenomics/chr_states/percentiles/{gene_type}_{state}.csv")
        chr_results[f"{gene_type}_{state}"] = df
        

In [29]:
# Plot the results
import matplotlib.pyplot as plt
import numpy as np

gbM_results = np.zeros((36, 300))
UM_results = np.zeros((36, 300))
teM_results = np.zeros((36, 300))



for (i, df) in chr_results.items():
    gene_type, state = i.split("_")
    for row in df.iter_rows(named=True):
        idx = int(row["percentile"]) +100
        if gene_type == "gbM":
            gbM_results[int(state) - 1, idx] = row["sites"]
        elif gene_type == "UM":
            UM_results[int(state) - 1, idx] = row["sites"]
        elif gene_type == "teM":
            teM_results[int(state) - 1, idx] = row["sites"]

gbM_results = pl.from_numpy(gbM_results, schema=[f"S{i+1}" for i in range(36)])
UM_results = pl.from_numpy(UM_results, schema=[f"S{i+1}" for i in range(36)])
teM_results = pl.from_numpy(teM_results, schema=[f"S{i+1}" for i in range(36)])

gbM_results.write_csv("/mnt/fast/epigenomics/chr_states/percentiles/gbM.csv")
UM_results.write_csv("/mnt/fast/epigenomics/chr_states/percentiles/UM.csv")
teM_results.write_csv("/mnt/fast/epigenomics/chr_states/percentiles/teM.csv")

In [31]:
def q(type="gbM"):
    return f"""
with percentile_windows as (
    select Count(*) as sites,
        FLOOR(
            (
                CASE
                    WHEN a.strand = '+' then ((c.start +c.end)/2 - a.start)
                    else (a.end - (c.start + c.end)/2
                )
            end * 100
        ) / (a.end - a.start
)
) as percentile, c.state as state
from chr_states c
    JOIN genes a ON c.chromosome = a.chromosome
where (c.start +c.end)/2 + (a.end - a.start
) >= a.start
and (c.start +c.end)/2 - (a.end - a.start
) <= a."end" and a.type = '{type}'
group by percentile,  state order by state, percentile
)
select sites, state,
    SUM(sites) OVER (
        ORDER BY percentile ROWS BETWEEN CURRENT ROW
            AND 4 FOLLOWING
    ) AS sites_sliding_sum,
    percentile
from percentile_windows order by state, percentile
"""

res = client.query_arrow(q("gbM"))
df = pl.DataFrame(res)

res = client.query_arrow(q("UM"))
df2 = pl.DataFrame(res)

res = client.query_arrow(q("teM"))
df3 = pl.DataFrame(res)


TypeError: DataFrame.pivot() missing 2 required positional arguments: 'index' and 'columns'

In [41]:
gbM = df.pivot(columns="state", values="sites", index="percentile" ).sort("percentile").fill_null(0)
display(gbM)

gbM.write_csv("/mnt/fast/epigenomics/chr_states/percentiles/gbM_pivot.csv")

percentile,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36
f64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64,u64
-100.0,23,12,14,23,24,29,21,15,15,21,14,17,6,19,4,20,12,17,12,14,20,13,14,19,30,13,10,32,30,29,5,2,2,2,0,0
-99.0,23,12,14,30,16,28,31,12,6,21,15,15,8,20,10,18,16,20,9,14,20,16,19,23,26,11,16,24,24,34,9,2,1,1,1,0
-98.0,33,15,19,15,28,24,18,10,11,14,13,20,8,21,12,20,17,18,22,9,21,15,15,20,30,16,9,21,26,29,7,11,8,1,0,0
-97.0,18,10,18,12,22,25,35,6,16,20,10,16,5,16,2,14,20,19,11,13,21,21,14,17,31,16,12,26,21,26,8,5,2,4,1,0
-96.0,21,9,13,23,26,39,21,11,10,15,13,19,12,16,4,15,24,20,7,15,25,12,11,17,24,12,10,17,26,22,8,5,2,0,0,0
-95.0,15,11,21,21,12,24,26,11,33,15,13,14,10,8,5,16,13,21,12,12,29,6,12,24,30,16,7,39,37,30,10,4,3,2,0,0
-94.0,32,9,13,17,29,30,20,19,11,17,10,14,7,26,4,17,24,17,12,18,17,14,16,19,34,14,10,24,37,30,8,4,4,1,0,1
-93.0,23,7,11,21,18,37,25,9,16,13,5,10,3,16,6,22,20,16,7,13,20,14,24,21,28,13,11,27,27,27,4,5,0,3,0,0
-92.0,21,10,15,25,28,25,20,10,15,7,12,24,9,20,5,23,16,19,7,11,20,5,11,28,16,19,8,35,35,27,7,5,3,2,1,0
